<style>
.mark-box {
  background-color: #f3f3f3;
  border-left: 6px solid #4CAF50;
  padding: 12px 24px;
  font-family: monospace;
  font-size: 14px;
}
</style>

<div class="mark-box">
<b>🏢 Empresa:</b> BICODE SAS<br>
<b>👨‍💻 Autor:</b> Ingeniero Fabian Izquierdo Perez<br>
<b>📅 Fecha:</b> 23/06/2025<br>
<b>🛠️ Tipo:</b> Data Processing<br>
<b>✉️ Email:</b> developer@bicode.co<br>
<b>📌 Descripción:</b><br>
Este notebook documenta la creación de <b>puntos de montaje seguros</b> en Azure Databricks sobre ADLS Gen2.<br><br>
Se aplican <b>buenas prácticas de seguridad</b> mediante el uso de:
<ul>
  <li>App Registrations con permisos mínimos necesarios</li>
  <li>Azure Key Vault para gestión segura de secretos</li>
  <li>Mounts configurados con scopes seguros</li>
</ul>
Esto permite establecer una arquitectura robusta, centralizada y segura para el acceso a los datos en el lakehouse.

</div>


## Configuration - Mount Points

### 1. _For this configuration the following services must be provisioned:_


- 1. #### _Creacion de un servicio en azure Key vault:_
<div style="text-align: center;">
  <img src="https://learn.microsoft.com/es-es/dotnet/azure/media/azure-key-vault.svg" alt="Key Vault" width="100"/>
</div>
- 2. #### _Creacion de un Azure storage Account:_
<div style="text-align: center;">
  <img src="https://learn.microsoft.com/es-es/dotnet/azure/media/storage-blobs.svg" alt="Key Vault" width="100"/>
</div>
- 3. #### _Creacion de un Azure Databricks:_ 
<div style="text-align: center;">
  <img src="https://www.databricks.com/sites/default/files/2023-03/azure-feature-1.jpg?v=1724844237" alt="Key Vault" width="100"/>
</div>

### 2. _Initial Configuration:_

- 1. Crear un Scope En Azure Databricks
> Una Opción es creandolo desde el servicio de databricks, utilizando "#secrets/createScope".  
> Ejemplo : https://adb-****************.10.azuredatabricks.net/#secrets/createScope  
> Este Scope creado debe ir configurado con el DNS Name del key vault donde se van a almacenar los secretos

- 2. Creacion de app registration.  
> https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/RegisteredApps  

- 3. Rol en cuenta de almacenamiento al app registration.  
> Rol = "Storage Blob Data Contributor"

- 4. Rol Sobre el Key Vault creado a la entidad administrada "AzureDatabricks".  
> Rol = "Key Vault Secrets User"

### 3. _Creation of secrets:_

En el Azure Key Vaul creado se deben crear los siguientes secretos:  
- 1. _ClienteId_ : Este codigo se obtiene desde el App Registration - Application (client) ID
- 2. _ClientSercret_ : Este codigo se obtiene desde Certificados y secretos de App Registration, Campo  "valor"
- 3. _TenantId_: Este codigo se obtiene desde el Key vault, variable Directory ID  
- 4. _sase_xxxxx_: Nombre del storage account



### 4. _Creating the mount point:_  
A continuacion se debe implementar el siguiente codigo:

In [0]:
# Relacion de los diferentes contenedores de montaje #
MOUNT_CONTAINER_OPERATION = "operacion"
MOUNT_CONTAINER_LANDING = "bronze"
MOUNT_CONTAINER_TRUSTED = 'silver'
MOUNT_CONTAINER_REFINED = 'gold'
MOUNT_CONTAINER_CATALOGING = 'maestro-negocio'
# Variables de los recretos almacenados en el Key Vault #
TENANT_KEY = "TenantId"
CLIENT_KEY = "ClienteId"
SECRET_KEY = "ClientSercret"
# Nombre del Scope creardo en databricks
ENVIRONMENT_SCOPE = 'StorageScope'
LAKE_ACCOUNT = dbutils.secrets.get(ENVIRONMENT_SCOPE, 'sase_xxxxx')
ALIAS_MOUNT = "XXXXX" #Nombre que se asigna para identificar el mount

configs = {"fs.azure.account.auth.type": "OAuth",
             "fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
             "fs.azure.account.oauth2.client.id": "id",
             "fs.azure.account.oauth2.client.secret": "secret",
             "fs.azure.account.oauth2.client.endpoint": "endpoint"}

In [0]:
# Method to create mount point for all the containers in Datalake
def mount_point(ALIAS_MOUNT,str_mount_point, str_datalake_point, str_key_client, str_key_secret, str_tenant_secret, str_Enviroment_Scope, configs):  
  
  configs["fs.azure.account.oauth2.client.id"] = dbutils.secrets.get(str_Enviroment_Scope, str_key_client)
  configs["fs.azure.account.oauth2.client.secret"] = dbutils.secrets.get(str_Enviroment_Scope, str_key_secret)
  configs["fs.azure.account.oauth2.client.endpoint"] = "https://login.microsoftonline.com/{0}/oauth2/token"\
  .format(dbutils.secrets.get(scope = str_Enviroment_Scope, key = str_tenant_secret))

  try:
    dbutils.fs.mount(
    source = "abfss://" + str_mount_point + "@"+str_datalake_point+".dfs.core.windows.net/",
    mount_point = f"/mnt/{ALIAS_MOUNT}/"+str_mount_point,
    extra_configs = configs)
  except Exception as e:
    errorMsg = str(e)
    if "already mounted" in errorMsg :
      print("{} already mounted. Run previous cells to unmount first".format(str_mount_point))
    else:
      raise e

In [0]:
#Se Ejecuta la funcion por cada contenedor que se desee crear un punto de montaje.
mount_point(ALIAS_MOUNT,MOUNT_CONTAINER_LANDING, LAKE_ACCOUNT, CLIENT_KEY, SECRET_KEY, TENANT_KEY, ENVIRONMENT_SCOPE, configs)
mount_point(ALIAS_MOUNT,MOUNT_CONTAINER_TRUSTED, LAKE_ACCOUNT, CLIENT_KEY, SECRET_KEY, TENANT_KEY, ENVIRONMENT_SCOPE, configs)

### 5. _Validation options:_  

In [0]:
# Comando para listar todos los ámbitos de secretos disponibles en tu workspace
secret_scopes = dbutils.secrets.listScopes()

# Imprimir los ámbitos
print("Ámbitos de secretos disponibles:")
for scope in secret_scopes:
    print(f"- {scope.name}")

Ámbitos de secretos disponibles:
- KV-Transversal
- storage_scope_db
- StorageScope


In [0]:
# Reemplaza 'nombre-de-tu-ambito' con el nombre real de tu ámbito de secretos
scope_name = "StorageScope" # Ejemplo

print(f"\nSecretos en el ámbito '{scope_name}':")
try:
    secrets_in_scope = dbutils.secrets.list(scope_name)
    for secret in secrets_in_scope:
        print(f"- {secret.key}")
except Exception as e:
    print(f"Error al listar secretos en el ámbito '{scope_name}': {e}")
    print("Asegúrate de que el ámbito existe y tienes los permisos de LECTURA adecuados.")


Secretos en el ámbito 'StorageScope':
- ADB-sasetransversal-ClienteId
- ADB-sasetransversal-ClientSercret
- ADB-sasetransversal-Name
- ADB-TenantId
- Key-Scope-ADB
- StorageScope


In [0]:
mounts = dbutils.fs.mounts()

for mount in mounts:
    print("Mount Name: ", mount.mountPoint)
    print("Source: ", mount.source)
    # print("Options: ", mount.options)
    print("--------------------------------------------------")
#dbutils.fs.unmount("/mnt/bronze")


Mount Name:  /databricks-datasets
Source:  databricks-datasets
--------------------------------------------------
Mount Name:  /Volumes
Source:  UnityCatalogVolumes
--------------------------------------------------
Mount Name:  /databricks/mlflow-tracking
Source:  databricks/mlflow-tracking
--------------------------------------------------
Mount Name:  /databricks-results
Source:  databricks-results
--------------------------------------------------
Mount Name:  /mnt/icbf/bronze
Source:  abfss://bronze@[REDACTED].dfs.core.windows.net/
--------------------------------------------------
Mount Name:  /databricks/mlflow-registry
Source:  databricks/mlflow-registry
--------------------------------------------------
Mount Name:  /Volume
Source:  DbfsReserved
--------------------------------------------------
Mount Name:  /volumes
Source:  DbfsReserved
--------------------------------------------------
Mount Name:  /
Source:  DatabricksRoot
--------------------------------------------------